# Домашнее задание (50 баллов)

В этом домашнем задании вы познакомитесь с основами NLP, научитесь обрабатывать тексты.

В местах, где используется `...` (elipsis), требуется заменить его на код.

Установим необходимые зависимости:

In [1]:
!pip install -U pip
!pip install nltk tqdm seqeval scikit-learn datasets numpy

In [2]:
from typing import List, Dict, Tuple, Callable

## Токенизация (15 баллов)

Токенизация - это процесс преобразования текста в набор токенов.
Наивная реализация разбивает текст по пробелам. Более умные реализации учитывают пунктуацию.

### Библиотека NLTK (2 балла)

Научимся работать с токенизацией NLTK, где уже [реализована](https://www.nltk.org/api/nltk.tokenize.html#nltk.tokenize.word_tokenize) работа с пунктуацией.

https://www.nltk.org/

In [3]:
import nltk
from nltk.tokenize import word_tokenize


# https://www.nltk.org/nltk_data/
nltk.download("punkt")
nltk.download('punkt_tab')
nltk.download('wordnet')


def tokenize(text: str, language: str = "english") -> List[str]:
    return word_tokenize(text, language='english')

assert tokenize("") == []
assert tokenize("Hello, world!") == ["Hello", ",", "world", "!"]
assert tokenize("EU rejects German call to boycott British lamb.") == ["EU", "rejects", "German", "call", "to", "boycott", "British", "lamb", "."]

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/vkuznetsov/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/vkuznetsov/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/vkuznetsov/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


### Нормализация (3 балла)

Добавим нормализацию после токенизации. Пробуем [лемматизацию](https://www.nltk.org/api/nltk.stem.wordnet.html#nltk.stem.wordnet.WordNetLemmatizer) , [стемминг](https://www.nltk.org/api/nltk.stem.snowball.html#nltk.stem.snowball.EnglishStemmer) и [юникод](https://docs.python.org/3/library/unicodedata.html#unicodedata.normalize) нормализацию. Напишем функцию, которая будет принимать на вход токен после токенизации, нормализовать в NFC юникод форму, переводит в нижний регистр, лемматизирует слово и, если слово не изменилось после лемматизации, применяет стемминг.


Создайте функцию `normalize`:
   - Функция `normalize` должна принимать строку `token` и возвращать нормализованный токен.
   - Примените к токену Unicode нормализацию с помощью `unicode_nfc_normalizer`.
   - Преобразуйте токен в нижний регистр.
   - Примените лемматизацию с помощью `lemmatizer`.
   - Если лемматизированный токен отличается от исходного, верните его. В противном случае, примените стемминг с помощью `stemmer` и верните результат.

In [4]:
import unicodedata

from functools import partial
from nltk.stem.snowball import EnglishStemmer
from nltk.stem.wordnet import WordNetLemmatizer


stemmer = EnglishStemmer()
lemmatizer = WordNetLemmatizer()
unicode_nfc_normalizer = partial(unicodedata.normalize, "NFC")


def normalize(token: str) -> str:
  """
  Нормализует токен, применяя Unicode нормализацию, преобразование в нижний регистр,
  лемматизацию и стемминг при необходимости.

  :param token: Токен для нормализации
  :return: Нормализованный токен
  """

  normalized = unicode_nfc_normalizer(token)
  normalized = normalized.lower()

  lemmatized = lemmatizer.lemmatize(normalized)
  if lemmatized != normalized:
    return lemmatized

  stemmed = stemmer.stem(normalized)
  return stemmed


test_tokens = ["Worlds", "churches", "Helping"]
assert [normalize(token) for token in test_tokens] == ["world", "church", "help"]

### Добавляем Словарь (10 баллов)

Современные токенайзеры не только разбивают строки на токены, но и преобразуют последовательность токенов в последовательность числел. Объединим функцию токенизации, нормализации и отображения из токенов в индексы в один объект токенайзера.

Напишите класс `Tokenizer` для токенизации и нормализации текста.

Построение словаря:
   - Создайте метод `_build_vocabulary`, который принимает список текстов `texts` и обновляет словарь токенов.
   - Для каждого текста:
     - Токенизируйте и нормализуйте текст.
     - Обновите счетчик вхождений слов.
   - Для каждого слова, которое встречается не менее `min_count` раз, добавьте слово в словарь `word2idx` и список `idx2word`.

Кодирование и декодирование:
   - Создайте метод `encode_word`, который принимает слово `word` и возвращает его индекс с применением нормализации.
   - Создайте метод `encode`, который принимает текст `text` и возвращает список индексов токенов.
   - Создайте метод `decode`, который принимает список индексов `input_ids` и возвращает текст, вставляя пробелы между токенами.

> Note: для функций, которые могут долго исполнятся (`_build_vocab`), рекомендуется использовать библиотеку tqdm.

In [5]:
from collections import Counter
from tqdm.notebook import tqdm


class Tokenizer:
    def __init__(
            self,
            texts: List[str],
            tokenize_fn: Callable[[str], List[str]] = tokenize,
            normalize_fn: Callable[[str], str] = lambda token: token,
            min_count: int = 1,
    ) -> None:
        """
        Инициализация токенизатора.

        :param texts: список текстов для построения словаря
        :param tokenize_fn: функция для токенизации текста
        :param normalize_fn: функция для нормализации токенов
        :param min_count: минимальное количество вхождений слова для включения в словарь
        """
        self.min_count = min_count
        self.tokenize = tokenize_fn
        self.normalize = normalize_fn
        self.word2idx = {"<PAD>": 0, "<BOS>": 1, "<EOS>": 2, "<UNK>": 3}
        self.unk_token_id = 3
        self.idx2word = ["<PAD>", "<BOS>", "<EOS>", "<UNK>"]
        self.word2count = Counter()
        self._build_vocabulary(texts)

        # for ngram
        self.bos_token = 1
        self.eos_token = 2
        self.special_token_ids = (self.bos_token, self.eos_token)

    def _build_vocabulary(self, texts: List[str]):
        """
        Построение словаря на основе списка текстов.

        :param texts: список текстов
        """

        for text in tqdm(texts):
          for token in self.tokenize(text):
            # normalized token
            nt = self.normalize(token)
            self.word2count[nt] += 1

            if self.word2count[nt] >= self.min_count and nt not in self:
              # add token
              self.idx2word.append(nt)
              self.word2idx[nt] = len(self.idx2word) - 1

    def encode_word(self, text: str) -> int:
        """
        Кодирование слова в индекс с применением нормализации.

        :param text: слово
        :return: индекс слова
        """
        nt = self.normalize(text)
        encoded = self.word2idx.get(nt, self.unk_token_id)

        return encoded

    def encode(self, text: str) -> List[int]:
        """
        Кодирование текста в набор индексов.

        :param text: текст
        :return: набор индексов токенов
        """
        encoded = []

        for token in self.tokenize(text):
          token_enc = self.encode_word(token)
          encoded.append(token_enc)

        return encoded

    def decode(self, input_ids: List[int]) -> str:
        """
        Декодирование набора индексов в текст. Вставляет пробел между декодированнми токенами.

        :param input_ids: набор индексов токенов
        :return: текст
        """
        result = []

        for idx in input_ids:
          # Handle incorrect indexes
          if idx < 0 or idx >= len(self.idx2word):
            idx = self.unk_token_id

          result.append(self.idx2word[idx])

        return " ".join(result)

    def __len__(self) -> int:
        """
        Возвращает количество уникальных токенов в словаре.

        :return: количество уникальных токенов
        """
        return len(self.word2idx)

    def __contains__(self, item: str) -> bool:
        """
        Проверяет, содержится ли слово в словаре.

        :param item: слово
        :return: True, если слово содержится в словаре, иначе False
        """
        return item in self.word2idx

    def __str__(self):
        """
        Возвращает строковое представление словаря.

        :return: строковое представление словаря
        """
        return str(self.word2idx)

In [6]:
corpus = ["Hello, world!", "I love Python!"]

tokenizer = Tokenizer(corpus, min_count=1)
encoded = tokenizer.encode("Hello, Python! I love you")
assert tokenizer.decode(encoded) == "Hello , Python ! I love <UNK>"

tokenizer = Tokenizer(corpus, normalize_fn=normalize)
encoded = tokenizer.encode("Hello, Python! I loved you")
assert tokenizer.decode(encoded) == "hello , python ! i love <UNK>"

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

## TF-IDF (20 баллов)


### Класс TFIDF (10 баллов)

Создайте класс `TFIDF` для вычисления TF-IDF значений. Формулы для подсчёта TF и IDF можно выбрать [тут](https://en.wikipedia.org/wiki/Tf%E2%80%93idf).

Обучение модели должно осуществляться с помощью метода `fit`, который принимает список строк `docs` и обучает модель на этом корпусе, вызывая метод `add_doc` для каждого документа.

Предсказание TF-IDF значений:
   - Создайте метод `predict`, который принимает список строк `docs` и возвращает матрицу TF-IDF значений.
   - Для каждого документа:
     - Токенизируйте документ.
     - Вычислите TF для каждого термина.
     - Вычислите IDF для каждого термина.
     - Заполните матрицу TF-IDF значений.
   - Нормализуйте строки матрицы, чтобы сумма значений в каждой строке была равна 1.

> Важно! Не забудьте убрать `<UNK>` токен во  время подсчёта TF-IDF 

Для функций, которые могут долго исполнятся (`fit`, `predict`), рекомендуется использовать библиотеку tqdm.

In [7]:
from collections import Counter
from typing import List
from tqdm.notebook import tqdm
import numpy as np
import math


class TFIDF:
    def __init__(self, tokenizer: Tokenizer, default_idf = 1.0) -> None:
        """
        Инициализация TFIDF.

        :param tokenizer: токенизатор для преобразования текста в токены
        :param default_idf: значение IDF для неизвестных токенов
        """
        self.tokenizer = tokenizer
        self.default_idf = default_idf
        self.reset()

    def reset(self) -> None:
        self.num_docs = 0
        self.term2num_docs = [0 for _ in self.tokenizer.word2idx]  # для подсчёта IDF        

    @property
    def vocab_size(self) -> int:
        """
        Возвращает размер словаря.

        :return: размер словаря
        """
        return len(self.tokenizer)

    def add_doc(self, doc: str) -> None:
        """
        Добавляет документ в модель TFIDF.

        :param doc: документ для добавления
        """
        self.num_docs += 1
        
        # Calculate term presence in doc
        terms_added = set(self.tokenizer.encode(doc))
        for term in terms_added:
            self.term2num_docs[term] += 1
        
    def fit(self, docs: List[str]) -> None:
        """
        Обучает модель TFIDF на корпусе docs.

        :param docs: корпус для обучения
        """
        # reset before fit, easier to debug
        self.reset()

        for doc in tqdm(docs):
            self.add_doc(doc)

    def predict(self, docs: List[str]) -> np.ndarray:
        """
        Предсказывает TFIDF значения для списка документов.

        :param docs: список документов
        :return: матрица TFIDF значений
        """
        idf = np.array([self.idf(i) for i in range(len(self.term2num_docs))])
        matrix = np.zeros(shape=(len(docs), len(self.term2num_docs)))

        doc_lengths = []

        for doc_idx, doc in enumerate(tqdm(docs)):
            # get tokens except unknown
            doc_tokens = [t for t in self.tokenizer.encode(doc) if t != self.tokenizer.unk_token_id]

            doc_lengths.append(len(doc_tokens))
            
            # skip empty doc
            if not doc_tokens:
                continue

            tf_idf = matrix[doc_idx, :]
            
            # calculate token occurrence
            cnt = Counter(doc_tokens)
            for term, count in cnt.items():
                tf_idf[term] = count

        # for zero doc length, set doc length to 1 to avoid div by zero
        # it will not affect the results as all values in the row will be 0s
        doc_lengths_updated = [dl if dl > 0 else 1 for dl in doc_lengths]
        
        # calculate term frequency for a doc
        matrix /= np.array(doc_lengths_updated).reshape(-1,1)
            
        # calculate tf-idf for a doc
        matrix *= idf

        # normalize
        sum_per_doc = matrix.sum(axis=1,keepdims=True)
        
        # for zero rows - avoid div by zero
        sum_per_doc[sum_per_doc == 0] = 1
        matrix /= sum_per_doc

        return matrix

    def idf(self, term: int) -> float:
        """
        Вычисляет IDF (обратную частоту документа) для термина.

        :param term: термин
        :return: IDF значение
        """
        if term == self.tokenizer.unk_token_id:
            return self.default_idf

        df = self.term2num_docs[term]
        idf = math.log(self.num_docs / (df + 1)) + 1

        return idf

        # тестовый корпус из ячейки ниже содержит всего 2 текста
        # токен "test" появляется в обоих и в классической формуле
        # idf("test") == log(2 / 2) == 0
        # поэтому в домашнем задании рекомендуется использовать
        # сглаженный вариант подсчёта inverse document frequency smooth:
        # log(N / (n + 1)) + 1
        # https://en.wikipedia.org/wiki/Tf%E2%80%93idf#Inverse_document_frequency
        ...

Тесты были проверены для такой комбинации формул:

$$
\text{TF} = \frac{f_{t,d}}{\sum_{t' \in d} f_{t',d}}
$$
$$
\text{IDF} = \log{\frac{N}{1 + n_t}} + 1
$$

Если вы выбрали другие формулы для подсчёта, то можно поправить тесты соответственно.

> Можно расширить расширить класс для удобства и поэксперементировать с различными формулами для подсчёта TF и IDF при классификации IMDB датасета ниже.

In [8]:
corpus = ["test test", "not a test"]
tokenizer = Tokenizer(corpus)
tfidf = TFIDF(tokenizer)
tfidf.fit(corpus)

assert tfidf.vocab_size == 4 + 3
# 3 токена, один из которых <UNK>
vector = tfidf.predict(["a test string"])[0]
# tf("a") == tf("test") and idf("a") > idf("test")
assert vector[tfidf.tokenizer.word2idx["a"]] > vector[tfidf.tokenizer.word2idx["test"]]

vector = tfidf.predict(["not a test a string"])[0]
assert vector[tfidf.tokenizer.word2idx["a"]] > 2 * vector[tfidf.tokenizer.word2idx["test"]]
assert vector[tfidf.tokenizer.word2idx["a"]] == 2 * vector[tfidf.tokenizer.word2idx["not"]]

assert not np.any(tfidf.predict(["all tokens abscent from vocab should be zeros vector"]))

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

### Датасет (1 балл)

В качестве датасета вёзмём популярный [набор отзывов на фильмы с сайта IMDB](https://huggingface.co/datasets/stanfordnlp/imdb). Нужно предсказать является ли отзыв позитивным или негативным. Чтобы скачать датасет воспользуемся библиотекой `datasets` из экосистемы `HuggingFace`. Интерфейс датасета похож на словарь, доступ к разным частям осуществляется по названию ключа:

1. Тренировочная часть датасета: `imdb["train"]`
2. Тексты для тренировки: `imdb["train"]["text"]`
3. Лейблы для тренировки: `imdb["train"]["label"]`

In [9]:
from datasets import load_dataset


imdb = load_dataset("stanfordnlp/imdb")

### Тренируем Токенайзер (2 балла)

Используя `"train"` часть датасета, инициализируйте два токеназйера:
1. Не использующий нормализацию
2. Использующий функцию `normalize`, определённую выше

Сравним размер полученного словаря в обоих случаях.

In [10]:
tokenizer_without_norm = Tokenizer(imdb["train"]["text"])
tokenizer_with_norm = Tokenizer(imdb["train"]["text"], normalize_fn=normalize)

assert len(tokenizer_without_norm) > len(tokenizer_with_norm)

  0%|          | 0/25000 [00:00<?, ?it/s]

  0%|          | 0/25000 [00:00<?, ?it/s]

### Тренируем TF-IDF Модель (2 балла)

Теперь мы можем натренировать модель на датасете. Обучим две модели, которые будут использовать разные токенайзеры.

In [11]:
tfidf_without_norm = TFIDF(tokenizer_without_norm)
tfidf_without_norm.fit(imdb["train"]["text"])

  0%|          | 0/25000 [00:00<?, ?it/s]

In [12]:
tfidf_with_norm = TFIDF(tokenizer_with_norm)
tfidf_with_norm.fit(imdb["train"]["text"])

  0%|          | 0/25000 [00:00<?, ?it/s]

### Обучим Логистическую Регрессию (5 баллов)

В качестве входов в модель нужно использовать TF-IDF представления документов (`X_train`), в качестве лейблов - 0 и 1, обозначающие нужный класс (`Y_train`). Начнём с модели, которая использует нормализацию.

Используюя тестовый датасет и `logreg.predict` проверьте предсказания модели, вычислив accuracy - количество правильных предсказаний, делённое на количество входных примеров.

In [13]:
from sklearn.linear_model import LogisticRegression


X_train = tfidf_with_norm.predict(imdb["train"]["text"])
Y_train = imdb["train"]["label"]

logreg = LogisticRegression()
logreg.fit(X_train, Y_train)

X_test = tfidf_with_norm.predict(imdb["test"]["text"])
Y_test = imdb["test"]["label"]

preds = logreg.predict(X_test)
accuracy = (preds == Y_test).sum() / len(X_test)
print("Accuracy: ", accuracy)

  0%|          | 0/25000 [00:00<?, ?it/s]

  0%|          | 0/25000 [00:00<?, ?it/s]

Accuracy:  0.78052


Теперь обучим логистическую регрессию со второй TF-IDF моделью и сравним результаты:

In [14]:
from sklearn.linear_model import LogisticRegression


X_train = tfidf_without_norm.predict(imdb["train"]["text"])
Y_train = imdb["train"]["label"]

logreg_without_norm = LogisticRegression()
logreg_without_norm.fit(X_train, Y_train) 

X_test = tfidf_without_norm.predict(imdb["test"]["text"])
Y_test = imdb["test"]["label"]

preds = logreg_without_norm.predict(X_test)
accuracy = (preds == Y_test).sum() / 25000
print("Accuracy: ", accuracy)

  0%|          | 0/25000 [00:00<?, ?it/s]

  0%|          | 0/25000 [00:00<?, ?it/s]

Accuracy:  0.76624


### (Опционально) TfidfVectorizer

Можете изучть класс [TfidfVectorizer](https://scikit-learn.org/1.5/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html) из библиотеки scikit-learn и сравнить его со своей имплементацией, обучив логистическую регрессию с его помощью.

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.linear_model import LogisticRegression


tfidf_vectorizer = TfidfVectorizer(use_idf=True)

X_train = tfidf_vectorizer.fit_transform(imdb["train"]["text"])
Y_train = imdb["train"]["label"]

logreg = LogisticRegression()
logreg.fit(X_train, Y_train)

X_test = tfidf_vectorizer.transform(imdb["test"]["text"])
Y_test = imdb["test"]["label"]

preds = logreg.predict(X_test)
accuracy = (preds == Y_test).sum() / X_test.shape[0]

print("Accuracy: ", accuracy)

Accuracy:  0.88292


## $n$-граммные языковые модели (15 баллов)

### Расширяем Токенайзер (3 балла)

Перед созданием языковой модели, расширим токенизационный класс. Добавим два флага в сигнатуру метода `encode`, чтобы управлять добавлением служебных токенов во время токенизации. Существующий метод `decode` уже пропускает `<PAD>` токен, добавим флаг `skip_special_tokens` для пропуска всех специальных токенов.

In [16]:
class BoSTokenizerEoS(Tokenizer):
    def encode(self, text: str, add_bos: bool = True, add_eos: bool = False) -> List[int]:
        """
        Кодирование текста в набор индексов.

        :param text: текст
        :param add_bos: добавление begin-of-sentence токена в начало
        :param add_eos: добавление end-of-sentence токена в конец
        :return: набор индексов токенов
        """
        encoded = super().encode(text)

        if add_bos:
            encoded.insert(0, self.bos_token)
        
        if add_eos:
            encoded.append(self.eos_token)

        return encoded

    def decode(self, input_ids: List[int], skip_special_tokens: bool = True) -> str:
        """
        Декодирование набора индексов в текст. Вставляет пробел между декодированнми токенами.

        :param input_ids: набор индексов токенов
        :param skip_special_tokens: пропуск специальных токенов во время декодирования.
        :return: текст
        """
        if skip_special_tokens:
            input_ids = [i for i in input_ids if i not in self.special_token_ids]

        return super().decode(input_ids)

### Создаём NGram Модель (12 баллов)

Создайте класс `NGramLanguageModel` для построения n-граммной языковой модели. В этом задании вы можете как опираться на предложенную структуру модели, так и сделать свою имплементацию.

Построение модели:
   - Создайте метод `_build_model`, который принимает список текстов `texts` и обновляет частоты n-грамм.
   - Для каждого текста:
     - Токенизируйте текст и добавьте токен `"<EOS>"` в конец.
     - Для каждого токена:
       - Определите префикс длиной `n-1`.
       - Обновите частоты n-грамм и частоты префиксов.

Генерация следующего токена:
   - Создайте метод `generate_next_token`, который принимает префикс `prefix` и возвращает следующий токен.
   - Преобразуйте префикс в кортеж.
   - Получите распределение частот для префикса.
   - Если распределение пустое, верните токен `"<UNK>"`.
   - Верните токен с наибольшей частотой.

Автодополнение текста:
   - Создайте метод `autocomplete`, который принимает текст `text` и максимальную длину `max_len`, и возвращает завершенный текст.
   - Токенизируйте текст.
   - Пока длина токенов меньше `max_len`:
     - Определите префикс длиной `n-1`.
     - Сгенерируйте следующий токен.
     - Добавьте токен в список токенов.
     - Если токен равен `"<EOS>"`, завершите генерацию.
   - Декодируйте и верните текст.

In [17]:
from collections import defaultdict


class NGramLanguageModel:
    def __init__(self, n: int, tokenizer: BoSTokenizerEoS, texts: List[str]):
        """
        Создание n-граммной языковой модели.

        :param n: порядок n-грамм
        :param vocabulary: словарь
        """
        assert n >= 2
        self.n = n
        self._context_length = self.n - 1 # added for convenience
        self.tokenizer = tokenizer
        self.frequencies = defaultdict(lambda: Counter())  # частота n-грамм
        self.frequencies_of_prefixes = Counter()  # сумма частот n-грамм для префиксов
        self._build_model(texts)


    def _build_model(self, texts: List[str]):
        """
        Построение модели на основе списка текстов.

        :param texts: список текстов
        """
        for text in texts:
            current_tokens = [self.tokenizer.bos_token] * self._context_length
            tokenized = self.tokenizer.encode(text, add_bos=False, add_eos=True) # Expected output: "some sentence here <eos>"

            # empty doc
            if not tokenized:
                continue

            for token in tokenized:
                prefix = tuple(current_tokens)

                # increase count of token for specified prefix
                self.frequencies[prefix][token] += 1

                # add token to context
                current_tokens.append(token)
                current_tokens = current_tokens[1:]

    def generate_next_token(self, prefix: List[int]) -> int:
        """
        Жадная генерация следующего токена по префиксу.

        :param prefix: префикс
        :return: следующий токен
        """
        # extend smaller context to n - 1
        pr = prefix.copy()
        while len(pr) < self._context_length:
            pr.insert(0, self.tokenizer.bos_token)

        # trim larger context to n - 1
        if len(pr) > self._context_length:
            pr = pr[-self._context_length:]

        token_freqs = self.frequencies.get(tuple(pr), None)
        if not token_freqs:
            return self.tokenizer.unk_token_id
        
        most_common_next_token, _ = token_freqs.most_common(1)[0]
        
        return most_common_next_token

    def autocomplete(self, text: str, max_len: int = 32, skip_special_tokens: bool = True) -> str:
        """
        Автоматическое дополнение текста.

        :param text: текст
        :param max_len: максимальная длина текста
        :param skip_special_tokens: пропуск специальных токенов во время декодирования.
        :return: завершенный текст
        """
        context = self.tokenizer.encode(text, add_bos=True)

        # last token will be EOS, so generate 1 token less than max to end with <EOS>
        max_tokens_to_generate = max_len - len(context) - 1
        
        # generate next tokens
        while max_tokens_to_generate > 0:
            next_token = self.generate_next_token(context)
            if next_token == self.tokenizer.eos_token:
                break
            
            max_tokens_to_generate -= 1
            context.append(next_token)
        
        # add EOS
        context.append(self.tokenizer.eos_token)
        result = self.tokenizer.decode(context, skip_special_tokens)
        
        return result

In [18]:
corpus = ["Hello, world!", "I love Python!", "Hello, Python"]
tokenizer = BoSTokenizerEoS(corpus, min_count=1)
ngram_lm = NGramLanguageModel(2, tokenizer, corpus)

assert ngram_lm.autocomplete("Hello, Python", max_len=10) == "Hello , Python !"
assert ngram_lm.autocomplete("Hello, Python", max_len=10, skip_special_tokens=False) == "<BOS> Hello , Python ! <EOS>"

  0%|          | 0/3 [00:00<?, ?it/s]

# Комментарии

Если остались вопросы, на которые хочется получить ответ при ревью, это место для них:

# Вопросы
TfIdf. То, что своя имплементация медленнее, не удивительно, но почему accuracy так отличается?
- tfidf_without_norm      0.77
- tfidf_with_norm         0.78
- sclearn TfidfVectorizer 0.88

если алгоритм аналогичный не должно быть таких отличий


# Предложения
Я бы внес 2 правки в формулировки заданий

1. Обновить описание задания для TF-IDF, предложенная версия:

Создайте класс `TFIDF` для вычисления TF-IDF значений. Формулы для подсчёта TF и IDF можно выбрать [тут](https://en.wikipedia.org/wiki/Tf%E2%80%93idf).

Обучение модели должно осуществляться с помощью метода `fit`:
  - метод принимает список строк `docs` и обучает модель на этом корпусе, вызывая метод `add_doc` для каждого документа.
  - результатом тренировки является заполняется массива df (self.term2num_docs)

Предсказание TF-IDF значений:
   - Создайте метод `predict`, который принимает список строк `docs` и возвращает матрицу TF-IDF значений.
   - Для каждого документа:
     - Токенизируйте документ.
     - Вычислите TF для каждого термина.
     - Вычислите IDF для каждого термина.
     - Заполните матрицу TF-IDF значений.
   - Нормализуйте строки матрицы, чтобы сумма значений в каждой строке была равна 1.
   - Метод predict обрабатывает каждый переданный документ отдельно, без учета других документов. При этом df рассчитывается исключительно на основе документов, которые были обработаны методом fit.

> Важно! Не забудьте убрать `<UNK>` токен во  время подсчёта TF-IDF 

Для функций, которые могут долго исполнятся (`fit`, `predict`), рекомендуется использовать библиотеку tqdm.


2. BoSTokenizerEoS.decode
:param skip_special_tokens: пропуск специальных токенов во время декодирования.
не описано, какие токены специальные. Указать явно надо : EOS + BOS

Если промотать вниз то понятно становится, но для выполнения задания не должно требоваться пролистать ноутбук вниз